## Standard API of Mapping Types
`dictionary` can be considered as a subclass of `collection.Mapping` and `collection.MutableMapping`

In [1]:
from collections import abc

my_dict = {}
isinstance(my_dict, abc.Mapping), isinstance(my_dict, abc.MutableMapping)

(True, True)

In [2]:
# Build dictionaries
a = dict(one=1, two=2, three=3)
b = {'three': 3, 'two': 2, 'one': 1}
c = dict([('two', 2), ('one', 1), ('three', 3)])
d = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e # All these dictionaries are considered equal even if the order of the keys is not the same

True

In [3]:
print(list(a.keys()))
print(c)
print(c.popitem()) # removes and returns the last key-value pair added to the dict
print(c)

['one', 'two', 'three']
{'two': 2, 'one': 1, 'three': 3}
('three', 3)
{'two': 2, 'one': 1}


### Dict Comprehensions

In [4]:
dial_codes = [
    (880, 'Bangladesh'),
    (55,  'Brazil'),
    (86,  'China'),
    (91,  'India'),
    (62,  'Indonesia'),
    (81,  'Japan'),
    (234, 'Nigeria'),
    (92,  'Pakistan'),
    (7,   'Russia'),
    (1,   'United States'),
]

country_dial = {country: code for country, code in dial_codes}
country_dial

{880: 'Bangladesh',
 55: 'Brazil',
 86: 'China',
 91: 'India',
 62: 'Indonesia',
 81: 'Japan',
 234: 'Nigeria',
 92: 'Pakistan',
 7: 'Russia',
 1: 'United States'}

### Useful Mapping Methods
1. `dict.update(m)`:
    - First checks whether `m` has a keys method and, if it does, assumes it is a mapping. 
    - Otherwise, update() falls back to iterating over `m`, assuming its items are (key, value) pairs.

In [5]:
a = dict(one=1, two=2, three=3)
a.update({'four': 4, 'five':5}) # update a Mapping
print(a)

{'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5}


In [6]:
a.update([('five', 4), ('four', 5)]) # iterate over `m`
print(a)

{'one': 1, 'two': 2, 'three': 3, 'four': 5, 'five': 4}


2. `dict.setdefault(key, default)` equivalent to 
    ```python
    if key not in dict.keys():
        dict[key] = default
    ```


In [7]:
b = {'one': [1, 2, 3]}
try :
    b['two'].append(2)
except KeyError:
    print("Python Intepreter don't know the type of b['two'], so not able to append")
    b.setdefault('two', []).append(2)
    b.setdefault('one', []).append(4) # don't change the existing value
    print(b)

Python Intepreter don't know the type of b['two'], so not able to append
{'one': [1, 2, 3, 4], 'two': [2]}


## Mapping with Flexible Key Lookup
Here are another two solutions to deal with missing keys:

1. uses `dict = collections.defaultdict(default_factory)`:

 `dict[k]` (actually `dict.__getitem__(k)`) will call `default_factory` to create a default value

In [8]:
import collections

c = collections.defaultdict(list) # value is an empty list by default
c['one'].append(1)
c['two'] = 4
c

defaultdict(list, {'one': [1], 'two': 4})

2. subclass `dict` and use `__missing__` method

In [9]:
#class myDict(dict):
# better to subclass `UserDict`, will talk about it later
class myDict(collections.UserDict):
    def __missing__(self, key):
        return 0
d = myDict([('one', 1), ('two', 2)])
d['two'], d['three'], 'three' in d.keys()

(2, 0, False)

## Variations of `dict`
- `collections.defaultdict` already covered
- `collections.OrderedDict`: Maintains keys in insertion order, allowing iteration over items in a predictable order. The `popitem` method of an `OrderedDict` pops the last item by default, but if called as `my_odict.popitem(last=False)`, it pops the first item added. Now that the built-in dict also keeps the keys ordered since Python 3.6, the main reason to use OrderedDict is writing code that is backward-compatible with earlier Python versions and check the order between two dicts.


In [10]:
odict1 = collections.OrderedDict([('One', 1), ('Two', 2)])
odict2 = collections.OrderedDict([('Two', 2), ('One', 1)])
d = {'Two' : 2, 'One' : 1}
odict1 == d, odict2 == d, odict1 == odict2

(True, True, False)

- `collections.ChainMap`: Holds a list of mappings that can be searched as one. The lookup is performed on each mapping in order, and succeeds if the key is found in any of them. This is useful to interpreters for languages with nested scopes, where each mapping represents a scope context.

In [11]:
english = {'One' : 1, 'Two' : 2}
german = {'Eins' : 1, 'Zwei' : 2}
num = collections.ChainMap(english, german)
num['Eins']

1

- `collections.Counter`: A mapping that holds an integer count for each key. Updating an existing key adds to its count. 

In [12]:
ct = collections.Counter('abababababbaabbabababab')
print(ct)
ct.update('adccdcd')
print(ct)
ct.most_common(3)

Counter({'b': 12, 'a': 11})
Counter({'a': 12, 'b': 12, 'd': 3, 'c': 3})


[('a', 12), ('b', 12), ('d', 3)]

## Building custom mappings
### Why is subclass `collections.UserDict`?
- `dict` has some implementation shortcuts that end up forcing us to override methods that we can just inherit from UserDict with no problems.
- object of `UserDict` has an internal `dict` instance, called `data`

In [13]:
import collections

class StrKeyDict(collections.UserDict):
    # deal with the keys whose str version is already saved in dict
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]

    def __contains__(self, key):
        return str(key) in self.data # dict instance inside the object
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

d = StrKeyDict()
isinstance(d, collections.abc.Mapping), isinstance(dict, collections.abc.MutableMapping)

(True, False)

`UserDict` subclasses `abc.MutableMapping` and `abc.MutableMapping`. There are still some methods we need to notice:
- `MutableMapping.update`: load instances from other mappings, call `__setitem__`
- `Mapping.get`: can be directly inherited

## Immutable Mappings
The mapping types provided by the standard library are all mutable, but you may need to guarantee that a user cannot change a mapping by mistake. 

- `types.MappingProxyType` returns a read-only `mappingproxy` instance given a mapping
- `mappingproxy` is also dynamic, any changes to the original will be reflected in the respective `mappingproxy` object

In [14]:
from types import MappingProxyType
d = {1:'A'}
d_proxy = MappingProxyType(d)
try:
    d_proxy[2] = 'x' # `mappingproxy` object does not support item assignment
except TypeError:
    print(d_proxy)
d[2] = 'B'
print(d_proxy) # dynamically reflected

{1: 'A'}
{1: 'A', 2: 'B'}


## Dictionary Views
The `dict` instance methods `.keys()`, `.values()`, and `.items()` return instances of classes called `dict_keys`, `dict_values`, and `dict_items`, respectively.

They are all dynamic

In [15]:
d = dict(a=10, b=20, c=30)
values = d.values()
print(values)
print(len(values))
print(list(values))
print(list(reversed(values)))
try:
    values[0]
except TypeError:
    print('`dict_values` object is not subscriptable')

dict_values([10, 20, 30])
3
[10, 20, 30]
[30, 20, 10]
`dict_values` object is not subscriptable


In [16]:
d['z'] = 260
values # dynamic

dict_values([10, 20, 30, 260])

## Set Theory
- A `set` is a collection of unique objects. A basic use case is removing duplication
- `frozenset` is its **immutable** sibling
- Set elements must be hashable. The `set` type is not hashable, so you can’t build a `set` with nested `set` instances. But `frozenset` is hashable, so you can have `frozenset` elements inside a `set`.

In [17]:
l = ['a', 'a', 'b', 'b', 'c']
set(l) # do not preserve the order

{'a', 'b', 'c'}

### Intersection
For two sets `a` and `b`:
- `a | b` returns the union
- `a & b` returns the intersection
- `a - b` returns the difference
- `a ^ b` returns the symmetric difference, equivalent to `(a - b) | (b - a)`

In [18]:
# Only works for sets
s1 = {'a', 'b'}
s2 = {'a', 'c'}
s1 | s2, s1 & s2, s1 - s2, s1 ^ s2

({'a', 'b', 'c'}, {'a'}, {'b'}, {'b', 'c'})

In [19]:
# Works for all iterable types
l1 = ['a', 'b']
l2 = ['a', 'c']

print(set(l1) & set(l2))
print(set(l1).intersection(l2)) # Usually only transform the smaller one to a set

{'a'}
{'a'}


### Set Literals
There’s no literal notation for the empty `set`, so we must remember to write `set()`.

In [20]:
s = {1}
print(type(s))
d = {}
print(type(d))
es = set()
print(type(es))

<class 'set'>
<class 'dict'>
<class 'set'>


There is no special syntax to represent `frozenset` literals—they must be created by calling the constructor.

In [21]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

## Set Comprehensions

In [22]:
{ord(i) for i in "WKW快去写代码" if ord(i) > 128}

{20195, 20889, 21435, 24555, 30721}

## Internals of sets and dicts
The lookup time for keys in dicts or sets is negligible, regardless of the size
### Set hash tables under the hood
The core data structure of a Python `set` is a hash table with at least 8 rows. Traditionally, the rows in hash table are called buckets

### Hashes and equality
- If two objects compare equal, their hash codes must also be equal, even if they are of different types
- There are 2^64 possible hash codes, but objects that are different may have the same hash code

In [23]:
hash(1) == hash(1.0)

True

### The hash table algorithm
How does `workdays = {'Mon', 'Tue', 'Wed', 'Thu', 'Fri'}` work? As follows

![hashSet](./Images/hashSet.png)

0. Initialize hash table

    The hash table for a `set` starts with **8 empty buckets**. As elements are added, Python makes sure **at least 1/3 of the buckets are empty**—doubling the size of the hash table when more space is needed. The hash code field of each bucket is initialized with -1, which means “no hash code”

In [24]:
hash_table = [(-1,'null') for i in range(8)]
hash_table

[(-1, 'null'),
 (-1, 'null'),
 (-1, 'null'),
 (-1, 'null'),
 (-1, 'null'),
 (-1, 'null'),
 (-1, 'null'),
 (-1, 'null')]

1. Compute the hash code for the first element and put the element in the empty bucket

    Given the literal `{'Mon', 'Tue', 'Wed', 'Thu', 'Fri'}`, Python gets the hash code for the first element, `'Mon'`. Use the remainder of its hash code divided by the number of buckets (8 here) as the index

In [25]:
code = hash('Mon') 
i = code % 8
hash_table[i] = (code, 'Mon')
hash_table

[(-1, 'null'),
 (-1, 'null'),
 (-1, 'null'),
 (-1, 'null'),
 (-1913363215878030988, 'Mon'),
 (-1, 'null'),
 (-1, 'null'),
 (-1, 'null')]

2. For remaining itens:

    if the remainder is already taken (index collision), and the hash code there is different from the new item, then Python probe the next bucket until we find an empty bucket. 

In [26]:
def push_set(table, lst):
    for item in lst:
        code = hash(item)
        index = code % len(table)
        while (table[index % len(table)][0] != -1):
            index += 1
            # if there is no empty bucket, extend not considered here
            if (index > 2 * len(table) - 2):
                print("Failed")
                return table
        index = index % len(table)
        table[index] = (code, item)
        print("Success")
    return table

l = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
hash_table = [(-1,'null') for i in range(8)]
hash_table = push_set(hash_table, l)
hash_table

Success
Success
Success
Success
Success


[(4328699346170196888, 'Tue'),
 (-8761516211810743815, 'Wed'),
 (7169429547465764570, 'Fri'),
 (6965629236275275355, 'Thu'),
 (-1913363215878030988, 'Mon'),
 (-1, 'null'),
 (-1, 'null'),
 (-1, 'null')]

### Hash table usage in `dict`
`swimmers = {'Mon': 14, 'Tue': 12, 'Wed': 14, 'Thu': 11}`

#### Older version 
![hashDictOld](./Images/hashDictOld.png)

In [27]:
def push_dict_old(table, lst):
    for item in lst:
        code = hash(item[0])
        index = code % len(table)
        while (table[index % len(table)][0] != -1):
            index += 1
            # if there is no empty bucket, extend not considered here
            if (index > 2 * len(table) - 2):
                print("Failed")
                return table
        index = index % len(table)
        table[index] = (code, item)
        print("Success")
    return table

hash_table = [(-1,'null') for i in range(8)]
l = [('Mon', 14), ('Tue', 12), ('Wed', 14), ('Thu', 11)]
push_dict_old(hash_table, l)
hash_table

Success
Success
Success
Success


[(4328699346170196888, ('Tue', 12)),
 (-8761516211810743815, ('Wed', 14)),
 (-1, 'null'),
 (6965629236275275355, ('Thu', 11)),
 (-1913363215878030988, ('Mon', 14)),
 (-1, 'null'),
 (-1, 'null'),
 (-1, 'null')]

#### Newer version: compact dict, **Preserves key insertion order**
![hashDictNew](./Images/hashDictNew.png)
Two tables in total, one tabel `entries` for all the key-value pairs **in the insertion order**, another table `indices` for the indices

0. set up `indices`
    
    The indices table is initially set up as an array of signed bytes, with 8 buckets, each initialized with -1 to signal “empty bucket”. 

In [28]:
indices = [-1 for i in range(8)]
indices

[-1, -1, -1, -1, -1, -1, -1, -1]

1. compute the hash code for the first key, add the remainder to `indices` and put the key-value pair in `entries` 

In [29]:
l = [('Mon', 14), ('Tue', 12), ('Wed', 14), ('Thu', 11)]
code = hash(l[0][0])
index = code % len(indices)
indices[index] = 0 # 0 means the first key
entries = []
entries.append([code, l[0]])
indices, entries

([-1, -1, -1, -1, 0, -1, -1, -1], [[-1913363215878030988, ('Mon', 14)]])

2. For remaining items:
    - No collision: put the key-value pair in `entries` and update the `indices`
    - collision: calculate `index`(remainder), if `entries[indices[index]][0] != code`, `index += 1`
    

In [30]:
def push_dict_new(indices, entries, lst):
    for item in lst:
        code = hash(item[0])
        index = code % len(indices)
        while(indices[index%len(indices)] != -1):
            index += 1
            # if there is no empty bucket, extend not considered here
            if (index > 2 * len(indices) - 2):
                print("Failed")
                return indices, entries
        index = index % len(indices)
        indices[index] = len(entries)
        entries.append([code, item])
        print("Success")
    return indices, entries

l = [('Mon', 14), ('Tue', 12), ('Wed', 14), ('Thu', 11)]
indices = [-1 for i in range(8)]
entries = []
indices, entries = push_dict_new(indices, entries, l)
indices, entries

Success
Success
Success
Success


([1, 2, -1, 3, 0, -1, -1, -1],
 [[-1913363215878030988, ('Mon', 14)],
  [4328699346170196888, ('Tue', 12)],
  [-8761516211810743815, ('Wed', 14)],
  [6965629236275275355, ('Thu', 11)]])